In [9]:
import asyncio
from shazamio import Shazam
import yt_dlp
import nest_asyncio

nest_asyncio.apply()

async def download_audio(youtube_url: str, output_path: str, cookies_path: str):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': output_path,
        'noplaylist': True,         # Ensure only the single video is downloaded
        'overwrites': True,         # Overwrite the file if it already exists
        'cookies': cookies_path,    # Use cookies file
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

async def recognize_track(file_path: str):
    shazam = Shazam()
    out = await shazam.recognize(file_path)
    print(out)
    if 'track' in out:
        track = out['track']
        song_title = track.get('title', 'Unknown Title')
        artist_name = track.get('subtitle', 'Unknown Artist')
        print(f"Song: {song_title}, Artist: {artist_name}")
    else:
        print("Could not recognize the track.")

async def main():
    cookies_file = '../data/cookies.txt'                            # Path to your cookies file
    # youtube_url = 'https://www.youtube.com/shorts/lTqjbKZZ9vk'    # hard
    youtube_url = 'https://www.youtube.com/shorts/IykG5LsziGw'      # easy
    audio_file = '../data/video'
    await download_audio(youtube_url, audio_file, cookies_file)

    # audio_file = '../data/harder.mp3'
    
    await recognize_track(audio_file)

await main()

[youtube] Extracting URL: https://www.youtube.com/shorts/IykG5LsziGw
[youtube] IykG5LsziGw: Downloading webpage


[youtube] IykG5LsziGw: Downloading tv player API JSON
[youtube] IykG5LsziGw: Downloading ios player API JSON


ERROR: [youtube] IykG5LsziGw: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


DownloadError: ERROR: [youtube] IykG5LsziGw: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies